<a href="https://colab.research.google.com/github/daniel-gomm/document-corpus-generation-pipelines/blob/main/document-corpus-generation-pipelines/notebooks/classification/EvaluationIMRaDModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script where the IMRaD models are evaluated. Probably best to open this in Google Colab.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 6.6 MB/s 
     |████████████████████████████████| 895 kB 48.5 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 64.3 MB/s 
     |████████████████████████████████| 636 kB 63.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModelForSequenceClassification 

from transformers import BertForSequenceClassification
from transformers import BertModel, BertConfig
import random
import numpy as np
import operator

from transformers import AutoTokenizer, AutoModel
import json
import glob
import os
import csv
import nltk
import bs4
from bs4 import BeautifulSoup
from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline
from transformers import RobertaTokenizer
from transformers import RobertaConfig, RobertaModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
import json
import glob
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def evaluateIMRADFromJson(data, modelstring, modelname, finetunedModelPath,tokenizerName, roberta, cased): #params: data: evaluation data, modelstring: base model string to be used, modelName:modelName, toenizerName, roberta: whether roberta is used, cased: whether to use a cased tokenizer-->True:lower cased tokenizer
  label_dict = {}
  label_dict["intro"]=0
  label_dict["related"]=1
  label_dict["method"]=2
  label_dict["results"]=3
  label_dict["discussion"]=4
  #load tokenizer
  tokenizerString=modelstring
  if (roberta==True):
    tokenizer = tokenizerName.from_pretrained(tokenizerString)
    print("roberta")
  else: 
    tokenizer = tokenizerName.from_pretrained(tokenizerString,do_lower_case=cased)
    #print(cased)
  #load model, create pipeline which returns LABEL_0 to LABEL_4 based on section
  model = modelname.from_pretrained(modelstring, num_labels=5, output_attentions=False, output_hidden_states=False)
  model.load_state_dict(torch.load(finetunedModelPath, map_location=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')))        
  classificationPipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0)
  c = 5
  r = 5
  #Evaluation Array: Which catgeory was labelled as what?
  EvaluationArray = [ [0] * c for i in range(r) ]
  EvaluationArrayWithContext = [ [0] * c for i in range(r) ]
  EvaluationArrayWithSections= [ [0] * c for i in range(r) ]
  EvaluationArrayWithSectionsAfterContext= [ [0] * c for i in range(r) ]
  arrayLabels=[]
  arrayBase=[]
  arrayContext=[]
  arraySection=[]
  labelArray=[]

  counterForSections=0
  rightLabels=0
  for paperCounter in range(1,(len(data))):
    predictedLabels=[]
    predictedLabels2=[]
    predictedLabels3=[]

    helperArray=[]
    arrayLabels.append(labelArray)
    labelArray=[]
    idArray=[]
    sections=data[str(paperCounter)]

    if (bool(data[str(paperCounter)]))==True:
      for sectionCounter in range(0,len(sections)):
        if (len(sections[str(sectionCounter)]["text"].split())>150):
          print("error")
          sections[str(sectionCounter)]["text"]="error"
          continue
        
        predictedLabels.append(classificationPipeline(sections[str(sectionCounter)]["text"])[0]["label"]) #predict label
        labelArray.append(sections[str(sectionCounter)]["section"])

      for i in range(0,len(predictedLabels)):

        int1=0
        int2=0
        for key in label_dict:
          if (labelArray[i]==key):
            int1=label_dict[key]
        label=predictedLabels[i]
        #replace label with matching string, more intuitive
        if (label=="LABEL_0"):
          label="intro"     
        if (label=="LABEL_1"):
          label="related"            
        if (label=="LABEL_2"):
          label="method"           
        if (label=="LABEL_3"):
          label="results"           
        if (label=="LABEL_4"):
          label="discussion"
        predictedLabels[i]=label
        for key in label_dict:
          if (label==key):
            int2=label_dict[key]       
        EvaluationArray[int1][int2]=int(EvaluationArray[int1][int2])+1
      predictedLabels2=predictedLabels[:]
      predictedLabels3=predictedLabels[:]

      arrayBase.append(predictedLabels)
      for i in range(0,len(predictedLabels2)):
        if (i<(len(predictedLabels2)-2)):
          if (predictedLabels2[i]==predictedLabels2[i+2]):
            predictedLabels2[i+1]=predictedLabels2[i]

          int1=0
          int2=0

          for key in label_dict:
            if (labelArray[i]==key):
              int1=label_dict[key]
          label=predictedLabels2[i]
        for key in label_dict:
          if (label==key):
            int2=label_dict[key]       
        EvaluationArrayWithContext[int1][int2]=int(EvaluationArrayWithContext[int1][int2])+1
      arrayContext.append(predictedLabels2)

      most_common = max(predictedLabels, key = predictedLabels.count)
      for i in range(0,len(predictedLabels)):
        predictedLabels3[i]=most_common
        for key in label_dict:
          if (labelArray[i]==key):
            int1=label_dict[key]
        label=predictedLabels3[i]
        for key in label_dict:
          if (label==key):
            int2=label_dict[key]       
        EvaluationArrayWithSections[int1][int2]=int(EvaluationArrayWithSections[int1][int2])+1

      arraySection.append(predictedLabels3)


  rightLabels=EvaluationArray[0][0]+EvaluationArray[1][1]+EvaluationArray[2][2]+EvaluationArray[3][3]+EvaluationArray[4][4]
  rightLabelsIntroRelated=EvaluationArray[0][0]+EvaluationArray[0][1]+EvaluationArray[1][0]+EvaluationArray[1][1]+EvaluationArray[2][2]+EvaluationArray[3][3]+EvaluationArray[4][4]
  for i in EvaluationArray:
    for p in i:
      sectionCounter+=p
  #print statements give further information which categroy was labelled as what
  #print(f"Without Context:{rightLabels/sectionCounter}")
  #print(f"Without Context intro=related:{rightLabelsIntroRelated/sectionCounter}")
  #print(EvaluationArray)

  rightLabels=EvaluationArrayWithContext[0][0]+EvaluationArrayWithContext[1][1]+EvaluationArrayWithContext[2][2]+EvaluationArrayWithContext[3][3]+EvaluationArrayWithContext[4][4]
  rightLabelsIntroRelated=EvaluationArrayWithContext[0][0]+EvaluationArrayWithContext[0][1]+EvaluationArrayWithContext[1][0]+EvaluationArrayWithContext[1][1]+EvaluationArrayWithContext[2][2]+EvaluationArrayWithContext[3][3]+EvaluationArrayWithContext[4][4]
  sectionCounter=0
  for i in EvaluationArrayWithContext:
    for p in i:
      sectionCounter+=p
  #print(f"With Context:{rightLabels/sectionCounter}")
  #print(f"With Context, intro=related:{rightLabelsIntroRelated/sectionCounter}")
  #print(EvaluationArrayWithContext)

  rightLabels=EvaluationArrayWithSections[0][0]+EvaluationArrayWithSections[1][1]+EvaluationArrayWithSections[2][2]+EvaluationArrayWithSections[3][3]+EvaluationArrayWithSections[4][4]
  rightLabelsIntroRelated=EvaluationArrayWithSections[0][0]+EvaluationArrayWithSections[0][1]+EvaluationArrayWithSections[1][0]+EvaluationArrayWithSections[1][1]+EvaluationArrayWithSections[2][2]+EvaluationArrayWithSections[3][3]+EvaluationArrayWithSections[4][4]
  sectionCounter=0
  for i in EvaluationArrayWithSections:
    for p in i:
      sectionCounter+=p
  #print(f"With Sections:{rightLabels/sectionCounter}")
  #print(f"With Sections, intro=related:{rightLabelsIntroRelated/sectionCounter}")
  #print(EvaluationArrayWithSections)

  arrayLabels.append(labelArray)
  array=[arrayLabels,arrayBase,arrayContext,arraySection]
  return (array)

In [8]:
BertTokenizerName=BertTokenizer

BertStringUncased="bert-base-uncased"
BertStringCased="bert-base-cased"

ScibertStringUncased="allenai/scibert_scivocab_uncased"
ScibertStringCased="allenai/scibert_scivocab_cased"


BertModelName=BertForSequenceClassification
ScibertPath="allenai/scibert_scivocab_cased"

adressToSaveModelBertUncased="BertUncased"
adressToSaveModelBertCased="BertCased"

adressToSaveModelSciBertUncased="ScibertUncased"
adressToSaveModelSciBertCased="ScibertCased"

RobertaTokenizer=RobertaTokenizer
RobertaModel=RobertaForSequenceClassification
RobertaString="roberta-base"
adressToSaveModelRoberta="RobertaCased"

In [4]:
  label_dict = {}
  label_dict["intro"]=0
  label_dict["related"]=1
  label_dict["method"]=2
  label_dict["results"]=3
  label_dict["discussion"]=4

Get evaluation data, the finetuned Bert, Scibert and ROberta models from Dropbox

In [29]:
!wget -O bertmodel https://www.dropbox.com/s/d7qhpm3u9d602hp/BertCased5.model?dl=0
!wget -O scibertmodel https://www.dropbox.com/s/rruwth9f9jtuhre/ScibertCased5.model?dl=0
!wget -O robertamodel https://www.dropbox.com/s/ewjaccflj1elxo4/Roberta.model?dl=0
!wget -O evaldata https://www.dropbox.com/s/gbn80vxzgrrtylz/EvaluationDataUnarxive.json?dl=0


--2021-09-05 19:21:27--  https://www.dropbox.com/s/d7qhpm3u9d602hp/BertCased5.model?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/d7qhpm3u9d602hp/BertCased5.model [following]
--2021-09-05 19:21:28--  https://www.dropbox.com/s/raw/d7qhpm3u9d602hp/BertCased5.model
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc520fe8e258a22e23c48658447c.dl.dropboxusercontent.com/cd/0/inline/BVnRn-Ps49OcYd2fZqhy2SwkASiUSbQlOCXI8Mg7LBIU-7QZq8tN_OUeySMisflEtB4SRlWGD8UCftfL5ldAtG2FMZ06vnpFb0NKkLVCZzKQbxGvaeg40P4EqK_UG1Vl2Lp-neG6DfisECF7LFScyhXR/file# [following]
--2021-09-05 19:21:28--  https://uc520fe8e258a22e23c48658447c.dl.dropboxusercontent.com/cd/0/inline/BVnRn-Ps49OcYd2fZqhy2SwkASiUSbQlOCXI8Mg7LBIU-7QZq8tN_OUeySMisflEtB4SRlW

In [34]:
filename="/content/evaldata"
with open(filename, 'r') as json_file:
  data = json.load(json_file)
array=evaluateIMRADFromJson(data, BertStringCased, BertModelName, "/content/bertmodel", BertTokenizerName, False, False)
x=[]
y=[]
z=[]
q=[]
for a in array[0]:
  for b in a:
    x.append(b)
for a in array[1]:
  for b in a:
    y.append(b)
for a in array[2]:
  for b in a:
    z.append(b)
for a in array[3]:
  for b in a:
    q.append(b)
#there will be a warning about the BERT weights, this is expected
print("\nMicro F1 Scores:")
print(f"Uncased BERT {f1_score(x,y, average='micro')}")
print(f"Uncased BERT with simple context {f1_score(x,z, average='micro')}")
print(f"Uncased BERT with sections {f1_score(x,q, average='micro')}")



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b


Micro F1 Scores:
Uncased BERT 0.6394052044609665
Uncased BERT with simple context 0.7211895910780669
Uncased BERT with sections 0.8252788104089219


In [35]:
array=evaluateIMRADFromJson(data, ScibertStringCased, AutoModelForSequenceClassification , "/content/scibertmodel", BertTokenizerName, False, False)
x=[]
y=[]
z=[]
q=[]
v=[]
for a in array[0]:
  for b in a:
    x.append(b)
for a in array[1]:
  for b in a:
    y.append(b)
for a in array[2]:
  for b in a:
    z.append(b)
for a in array[3]:
  for b in a:
    q.append(b)

print("\nMicro F1 Scores:")
print(f"Uncased SciBERT {f1_score(x,y, average='micro')}")
print(f"Uncased SciBERT with simple context {f1_score(x,z, average='micro')}")
print(f"Uncased SciBERT with sections {f1_score(x,q, average='micro')}")

print("\nMicro F1 Scores, Recall and Precision for different categories for SciBERT with simple context:")

for i in label_dict:
  v=z[:]
  w=x[:]
  for j in range (0, len(z)):
    if x[j]!=i:
      w[j]="False"
    if x[j]==i:
      w[j]=i
      
    if z[j]!=i:
      v[j]="False"
    if z[j]==i:
      v[j]=i

  print(f"\nSection: {i}")
  print(f"F1 Score: {f1_score(w,v, pos_label=i,average='binary')}")
  print(f"Recall: {recall_score(w,v,pos_label=i, average='binary')}")
  print(f"Precision: {precision_score(w,v, pos_label=i,average='binary')}")


Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were


Micro F1 Scores:
Uncased SciBERT 0.7211895910780669
Uncased SciBERT with simple context 0.7881040892193308
Uncased SciBERT with sections 0.9107806691449816

Micro F1 Scores, Recall and Precision for different categories for SciBERT with simple context:

Section: intro
F1 Score: 0.7708333333333334
Recall: 0.7254901960784313
Precision: 0.8222222222222222

Section: related
F1 Score: 0.7962962962962964
Recall: 0.8113207547169812
Precision: 0.7818181818181819

Section: method
F1 Score: 0.8333333333333333
Recall: 0.9016393442622951
Precision: 0.7746478873239436

Section: results
F1 Score: 0.7619047619047619
Recall: 0.7692307692307693
Precision: 0.7547169811320755

Section: discussion
F1 Score: 0.7628865979381444
Recall: 0.7115384615384616
Precision: 0.8222222222222222


In [36]:
array=evaluateIMRADFromJson(data, RobertaString, RobertaForSequenceClassification, "/content/robertamodel", RobertaTokenizer, True, True)

x=[]
y=[]
z=[]
q=[]
for a in array[0]:
  for b in a:
    x.append(b)
for a in array[1]:
  for b in a:
    y.append(b)
for a in array[2]:
  for b in a:
    z.append(b)
for a in array[3]:
  for b in a:
    q.append(b)

print("\nMicro F1 Scores:")
print(f"Uncased RoBERTa {f1_score(x,y, average='micro')}")
print(f"Uncased RoBERTa with simple context {f1_score(x,z, average='micro')}")
print(f"Uncased RoBERTa with sections {f1_score(x,q, average='micro')}")


roberta


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie


Micro F1 Scores:
Uncased RoBERTa 0.6840148698884758
Uncased RoBERTa with simple context 0.758364312267658
Uncased RoBERTa with sections 0.9479553903345725
